## 1. 数据抓取

以腾讯新闻为例：
- 第一步，输入初始下载地址，通常指定某一新闻网页的导航页；
- 第二步，运行爬虫程序，即从初始网址抓取第一组网页链接；
- 第三步，利用正则表达式对url中的目录页地址和新闻内容页地址进行获取，进而得到新地址，按照规则生成后放进下载队列；
- 第四步，运行爬虫程序，从队列中取出新网页，完成新闻内容爬取；
- 第五步，当检测到下载队列为空时，则停止爬虫程序，否则重新运行第四步。

In [12]:
import requests
import json

lists=[]
url = 'https://i.match.qq.com/ninja/fragcontent?pull_urls=news_top_2018&callback=__jp0'
html = requests.get(url)
# print(html.text)
# print(json.loads((html.text)[6:-2]))
for i in json.loads((html.text)[6:-2]):
 # print(i)
    lists.append(i['title'])
    print('新闻标题为：\n{}\n'.format(i['title']))
    print('新闻链接为：\n{}\n----------\n'.format(i['url']))


新闻标题为：
一枝一叶总关情·诗之韵

新闻链接为：
https://new.qq.com/omn/20200517/20200517A0C3DY00.html
----------

新闻标题为：
这几年“悬崖村”经历了什么？

新闻链接为：
https://new.qq.com/omn/TWF20200/TWF2020051700474600.html
----------

新闻标题为：
助力残疾人脱贫攻坚故事

新闻链接为：
https://new.qq.com/omn/20200517/20200517A0FPDE00.html
----------

新闻标题为：
十九届中央第五轮巡视全部进驻

新闻链接为：
https://new.qq.com/omn/20200517/20200517A025XC00.html
----------

新闻标题为：
奋斗是对人生最长情告白

新闻链接为：
https://new.qq.com/omn/20200517/20200517A026SA00.html
----------

新闻标题为：
做好“六稳”工作落实“六保”任务 稳住经济基本盘

新闻链接为：
https://new.qq.com/omn/20200517/20200517A0HLOB00.html
----------

新闻标题为：
法国病毒学专家：“新冠病毒来自实验室”是不切实际的空想

新闻链接为：
https://new.qq.com/omn/20200517/20200517A0HIWZ00.html
----------

新闻标题为：
【中国稳健前行】沧海横流，愈加彰显伟大民族精神

新闻链接为：
https://new.qq.com/omn/TWF20200/TWF2020051700298000.html
----------

新闻标题为：
“曲速行动”恐欲速不达 特朗普疫苗工程危机四伏

新闻链接为：
https://new.qq.com/omn/20200517/20200517A0HE7I00.html
----------

新闻标题为：
特朗普很会给人洗脑 可惜都是伪科学

新闻链接为：
https://new.qq.com/omn/20200517/20200517A0HH3A00.html
----------


### 将所有新闻标题进行汇总

In [13]:
print(lists)
print(len(lists))

['一枝一叶总关情·诗之韵', '这几年“悬崖村”经历了什么？', '助力残疾人脱贫攻坚故事', '十九届中央第五轮巡视全部进驻', '奋斗是对人生最长情告白', '做好“六稳”工作落实“六保”任务 稳住经济基本盘', '法国病毒学专家：“新冠病毒来自实验室”是不切实际的空想', '【中国稳健前行】沧海横流，愈加彰显伟大民族精神', '“曲速行动”恐欲速不达 特朗普疫苗工程危机四伏', '特朗普很会给人洗脑 可惜都是伪科学']
10


## 2.分词
系统的自然语音处理模块是基于北京大学开发的中文语义开放平台`PKUSeg`实现的。

In [20]:
#分词
import pkuseg
seg = pkuseg.pkuseg()
# text = seg.cut('此处为待分词的句子')
text1 = seg.cut('特朗普很会给人洗脑 可惜都是伪科学')
print("text1分词结果：",text1)
text2 = seg.cut('特朗普：“新冠病毒来自实验室”是不切实际的空想')
print("text2分词结果：",text2)

text1分词结果： ['特朗普', '很', '会', '给', '人', '洗脑', '可惜', '都', '是', '伪科学']
text2分词结果： ['特朗普', '：', '“', '新冠', '病毒', '来自', '实验室', '”', '是', '不切实际', '的', '空想']


### 预处理：将无含义词、标点符号 去除

In [21]:
def yuchuli(list):
    if '：' in list:list.remove('：')
    if '“' in list:list.remove('“')
    if '”' in list:list.remove('”')
    if '“' in list:list.remove('“')
    if '是' in list:list.remove('是')
    if '的' in list:list.remove('的')
    if '【' in list:list.remove('【')
    if '】' in list:list.remove('】')
    if '，' in list:list.remove('，')
    if '？' in list:list.remove('？')
    if '“' in list:list.remove('“')
    if '”' in list:list.remove('”')
yuchuli(text1)
yuchuli(text2)
print(text1)
print(text2)

['特朗普', '很', '会', '给', '人', '洗脑', '可惜', '都', '伪科学']
['特朗普', '新冠', '病毒', '来自', '实验室', '不切实际', '空想']


## 3.新闻相似度计算

本文采用了余弦相似度进行度量。在利用余弦相似度计算两个向量的距离前，需要将两个向量进行`编码`。

In [22]:
#将两个list放在一个set中
word_set = set(text1).union(set(text2))
print(word_set)

{'可惜', '伪科学', '人', '给', '新冠', '特朗普', '来自', '洗脑', '会', '实验室', '病毒', '都', '不切实际', '空想', '很'}


In [23]:
#将上述set转换为dict，key为set中的词，value为set中词出现的位置
word_dict = dict()
i = 0
for word in word_set:
    word_dict[word] = i
    i += 1
print(word_dict)

{'可惜': 0, '伪科学': 1, '人': 2, '给': 3, '新冠': 4, '特朗普': 5, '来自': 6, '洗脑': 7, '会': 8, '实验室': 9, '病毒': 10, '都': 11, '不切实际': 12, '空想': 13, '很': 14}


In [24]:
#将text1和text2进行编码，将每个字转换为出现在set中的位置
text1_code = [word_dict[word] for word in text1]
print("编码后的新闻1：",text1_code)
text1_code = [0]*len(word_dict)
for word in text1:
    text1_code[word_dict[word]]+=1
print("新闻1-oneHot编码：",text1_code)
text2_code = [word_dict[word] for word in text2]
print("编码后的新闻2：",text2_code)
text2_code = [0]*len(word_dict)
for word in text2:
    text2_code[word_dict[word]]+=1
print("新闻2-oneHot编码：",text2_code)

编码后的新闻1： [5, 14, 8, 3, 2, 7, 0, 11, 1]
新闻1-oneHot编码： [1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1]
编码后的新闻2： [5, 4, 10, 6, 9, 12, 13]
新闻2-oneHot编码： [0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0]


In [25]:
# 余弦相似度计算
import math
sum = 0
sq1 = 0
sq2 = 0
for i in range(len(text1_code)):
    sum += text1_code[i] * text2_code[i]
    sq1 += pow(text1_code[i], 2)
    sq2 += pow(text2_code[i], 2)

try:
    result = round(float(sum) / (math.sqrt(sq1) * math.sqrt(sq2)), 2)
except ZeroDivisionError:
    result = 0.0
print("两则新闻的余弦相似度为：",result)

两则新闻的余弦相似度为： 0.13


## 4.数据可视化

### 首先对数据进行汇总

In [26]:
allwords = []
for i in lists:
    seg = pkuseg.pkuseg()
    words =seg.cut(i)
    allwords.append(words)
import operator
from functools import reduce
allwords=reduce(operator.add,allwords)
yuchuli(allwords)
print(allwords)

['一', '枝', '一', '叶', '总', '关情·诗', '之', '韵', '这几年', '悬崖村', '经历', '了', '什么', '助力', '残疾人', '脱贫', '攻坚', '故事', '十九', '届', '中央', '第五', '轮', '巡视', '全部', '进驻', '奋斗', '对', '人生', '最', '长情', '告白', '做好', '六', '稳', '工作', '落实', '六', '保', '”', '任务', '稳住', '经济', '基本盘', '法国', '病毒学', '专家', '“', '新冠', '病毒', '来自', '实验室', '”', '是', '不切实际', '空想', '中国', '稳健', '前行', '沧海横流', '愈加', '彰显', '伟大', '民族', '精神', '“', '曲速', '行动', '”', '恐', '欲速不达', '特朗普', '疫苗', '工程', '危机四伏', '特朗普', '很', '会', '给', '人', '洗脑', '可惜', '都', '是', '伪科学']


### 对词频进行统计

In [27]:
from collections import Counter
dic_cipin=Counter(allwords)
dic_cipin

Counter({'一': 2,
         '枝': 1,
         '叶': 1,
         '总': 1,
         '关情·诗': 1,
         '之': 1,
         '韵': 1,
         '这几年': 1,
         '悬崖村': 1,
         '经历': 1,
         '了': 1,
         '什么': 1,
         '助力': 1,
         '残疾人': 1,
         '脱贫': 1,
         '攻坚': 1,
         '故事': 1,
         '十九': 1,
         '届': 1,
         '中央': 1,
         '第五': 1,
         '轮': 1,
         '巡视': 1,
         '全部': 1,
         '进驻': 1,
         '奋斗': 1,
         '对': 1,
         '人生': 1,
         '最': 1,
         '长情': 1,
         '告白': 1,
         '做好': 1,
         '六': 2,
         '稳': 1,
         '工作': 1,
         '落实': 1,
         '保': 1,
         '”': 3,
         '任务': 1,
         '稳住': 1,
         '经济': 1,
         '基本盘': 1,
         '法国': 1,
         '病毒学': 1,
         '专家': 1,
         '“': 2,
         '新冠': 1,
         '病毒': 1,
         '来自': 1,
         '实验室': 1,
         '是': 2,
         '不切实际': 1,
         '空想': 1,
         '中国': 1,
         '稳健': 1,
         '前行': 1,

### 绘制词云图

In [28]:
value=[]
allwords=list(set(allwords))
for i in allwords:
    value.append(dic_cipin[i])
from pyecharts import WordCloud
wordcloud = WordCloud(width=1000, height=620)
wordcloud.add("", allwords, value, word_size_range=[20, 100])
wordcloud.render()
wordcloud